## Tucker Carlson Limited Dictionary Topic Analysis

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
from datetime import datetime
from dateutil.parser import parse
from dateutil import tz
import altair as alt

In [2]:
#pip install openpyxl 

In [3]:
data_tucker=pd.read_csv('Tucker_transcripts_2.17.23.tsv',sep='\t').drop(columns = ['Unnamed: 0'])
data_tucker['transcript'] = data_tucker['text'].drop(columns = ['text'])

#data.iloc[50,3]

In [4]:
#Need to include datetime cleanup in web scraping or data cleaning file for Maddow

data_maddow=pd.read_csv('Maddow_transcripts_longer.tsv',sep='\t').drop(columns = ['Unnamed: 0'])
data_maddow=data_maddow.drop_duplicates().reset_index(drop=True)

In [5]:
# removing any extra dates/characters due to the 'updated' timestamp on some transcripts
def keep_first_datetime(datetime_str):
    first_part = datetime_str.split("/")[0]
    return first_part.strip()

In [6]:
# convert timestamp to datetime object

def string_to_datetime(datetime_str):
    est_tz = tz.gettz('EST')
    datetime_obj = parse(datetime_str, fuzzy=True, tzinfos={ 'EST': est_tz })
    return datetime_obj
    

In [7]:
data_maddow.timestamp = [keep_first_datetime(timestamp) for timestamp in data_maddow.timestamp]
data_tucker.timestamp = [keep_first_datetime(timestamp) for timestamp in data_tucker.timestamp]

In [8]:
warnings.filterwarnings("ignore", category=UserWarning, module="dateutil")

data_maddow.timestamp = [string_to_datetime(timestamp) for timestamp in data_maddow.timestamp]
data_tucker.timestamp = [string_to_datetime(timestamp) for timestamp in data_tucker.timestamp]

In [9]:

import re

def create_dict(file_name):
    pew= pd.read_excel(file_name)

    for count, pewrow in enumerate(pew['primary dict']):
        pew['primary dict'][count] = re.split(',',pewrow)

    #seed_topic = pew['primary dict'].to_list()
    
    return pew


pew = create_dict('Pew Issue Polling 2022.xlsx')

C:\Users\Owner\AppData\Local\Temp\ipykernel_14556\599084700.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pew['primary dict'][count] = re.split(',',pewrow)


In [10]:
pew

,issue,"Mar. 1-31, 2022 Total",Repub,Dem,"Aug. 1-14, 2022 Total",Repub.1,Dem.1,"Oct. 10-16, 2022 Total",Repub.2,Dem.2,Shift in importance March - OctoberTotal,Repub.3,Dem.3,primary dict
0,Abortion,0.43,0.40,0.46,0.56,0.41,0.71,0.56,0.39,0.75,0.13,-0.01,0.29,"[abortion, anti abortion, pro choice, pro life..."
1,Education,0.59,0.53,0.66,0.58,0.52,0.64,0.64,0.60,0.66,0.05,0.07,0.00,"[education, schools, school, teachers, teacher..."
2,Immigration,0.49,0.68,0.34,0.48,0.66,0.31,0.54,0.76,0.36,0.05,0.08,0.02,"[immigration, migration, immigrants, migrants,..."
3,Violent crime,0.56,0.67,0.47,0.60,0.69,0.52,0.61,0.74,0.45,0.05,0.07,-0.02,"[crime, criminals, theft, violent crime, crimi..."
4,Health care,0.60,0.44,0.74,0.60,0.43,0.77,0.63,0.42,0.79,0.03,-0.02,0.05,"[health care, health insurance, uninsured, oba..."
5,Gun policy,0.54,0.55,0.54,0.62,0.52,0.71,0.57,0.56,0.62,0.03,0.01,0.08,"[guns, mass shooting, mass shootings, second a..."
6,Energy policy,0.59,0.62,0.58,0.53,0.51,0.56,0.61,0.63,0.63,0.02,0.01,0.05,"[energy policy, green new deal, renewable ener..."
7,Economy,0.78,0.90,0.68,0.77,0.90,0.67,0.79,0.92,0.65,0.01,0.02,-0.03,"[economy, economic, inflation, jobs, wages, un..."
8,Issues around race and ethnicity,0.35,0.14,0.54,0.35,0.15,0.54,0.34,0.13,0.52,-0.01,-0.01,-0.02,"[black lives matter, blm, blm protests, police..."
9,Supreme Court appointments,0.56,0.55,0.59,0.58,0.48,0.69,0.55,0.52,0.69,-0.01,-0.03,0.10,"[supreme court nomination, supreme court appoi..."


In [11]:
# Load the regular expression library

def process_text(df):
    # Remove punctuation
    df['text_processed'] = df['transcript'].map(lambda x: re.sub('[,\.!?"]', '', x))
    # Convert the titles to lowercase
    df['text_processed'] = df['text_processed'].map(lambda x: x.lower())
    
    return df

    

In [12]:
data_tucker = process_text(data_tucker)

data_maddow = process_text(data_maddow)

In [13]:
data_tucker

,url,timestamp,title,text,transcript,text_processed
0,https://www.foxnews.com/opinion/tucker-carlson...,2018-11-27 08:39:00,Tucker Carlson: Socialism with open borders is...,Democrats want to eliminate restrictions on im...,Democrats want to eliminate restrictions on im...,democrats want to eliminate restrictions on im...
1,https://www.foxnews.com/opinion/tucker-carlson...,2018-11-28 07:16:00,"Tucker Carlson: For the crime of forgetting, J...",Jerome Corsi addresses charges against him in ...,Jerome Corsi addresses charges against him in ...,jerome corsi addresses charges against him in ...
2,https://www.foxnews.com/opinion/tucker-carlson...,2018-11-29 11:12:00,Tucker Carlson: The word 'Russia' eliminates a...,"On Wednesday, Tucker Carlson exposes the flaws...","On Wednesday, Tucker Carlson exposes the flaws...",on wednesday tucker carlson exposes the flaws ...
3,https://www.foxnews.com/opinion/tucker-carlson...,2018-11-30 09:21:00,Tucker Carlson: The Mueller probe and what we ...,Michael Cohen pleads guilty to lying to Congre...,Michael Cohen pleads guilty to lying to Congre...,michael cohen pleads guilty to lying to congre...
4,https://www.foxnews.com/opinion/tucker-carlson...,2018-12-04 08:08:00,Tucker Carlson: The Mueller probe continues to...,"On Monday, Tucker Carlson talks to Jerome Cors...","On Monday, Tucker Carlson talks to Jerome Cors...",on monday tucker carlson talks to jerome corsi...
...,...,...,...,...,...,...
808,https://www.foxnews.com/opinion/tucker-carlson...,2023-02-08 22:25:00,TUCKER CARLSON: The lies 'just kept coming' du...,Fox News host Tucker Carlson reacts to Presid...,Fox News host Tucker Carlson reacts to Presid...,fox news host tucker carlson reacts to presid...
809,https://www.foxnews.com/opinion/tucker-carlson...,2023-02-09 22:19:00,TUCKER CARLSON: Self-defense is becoming illeg...,WARNING: Graphic footage—Fox News host Tucker ...,WARNING: Graphic footage—Fox News host Tucker ...,warning: graphic footage—fox news host tucker ...
810,https://www.foxnews.com/opinion/tucker-carlson...,2023-02-13 22:23:00,"TUCKER CARLSON: Food, water, energy and infras...",Fox News host Tucker Carlson reacts to leaders...,Fox News host Tucker Carlson reacts to leaders...,fox news host tucker carlson reacts to leaders...
811,https://www.foxnews.com/opinion/tucker-carlson...,2023-02-14 22:51:00,TUCKER CARLSON: Mayor Pete is completely incom...,Fox News host Tucker Carlson calls out the Bid...,Fox News host Tucker Carlson calls out the Bid...,fox news host tucker carlson calls out the bid...


In [14]:
#add empty lists to all the issue columns so they can easily be extended
def lists(df, pew):
    for issue in pew['issue']: 
        df[issue] = np.empty((len(df), 0)).tolist()
    return df

data_tucker = lists(data_tucker, pew)

data_maddow =lists(data_maddow, pew)

In [15]:
data_maddow

,url,timestamp,transcript,text_processed,Abortion,Education,Immigration,Violent crime,Health care,Gun policy,...,Issues around race and ethnicity,Supreme Court appointments,Size and scope of federal government,Climate change,Voting policies,Foreign policy,The coronavirus outbreak,Future of Democracy in the country,Investigations into Trump,Investigations into Biden
0,https://www.msnbc.com/transcripts/rachel-maddo...,2022-10-04 01:00:00+00:00,"SummaryWith public trust at a low, the Supreme...",summarywith public trust at a low the supreme ...,[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
1,https://www.msnbc.com/transcripts/rachel-maddo...,2022-09-27 01:00:00+00:00,"Summary ""American Psychosis"" examines the rise...",summary american psychosis examines the rise o...,[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2,https://www.msnbc.com/transcripts/rachel-maddo...,2022-09-13 01:00:00+00:00,"Summary Interview with Geoffrey Berman, the U....",summary interview with geoffrey berman the us ...,[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
3,https://www.msnbc.com/transcripts/rachel-maddo...,2022-08-30 01:00:00+00:00,"SummaryThe ODNI is going to assess ""potential ...",summarythe odni is going to assess potential r...,[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
4,https://www.msnbc.com/transcripts/rachel-maddo...,2022-08-23 01:00:00+00:00,"SummaryInterview with Dr. Anthony Fauci, direc...",summaryinterview with dr anthony fauci directo...,[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,https://www.msnbc.com/transcripts/transcript-r...,2021-06-04 01:00:00+00:00,SummaryRepublicans in multiple states have exp...,summaryrepublicans in multiple states have exp...,[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
296,https://www.msnbc.com/transcripts/transcript-r...,2021-06-03 01:00:00+00:00,SummaryFBI attributes JBS cyberattack to Russi...,summaryfbi attributes jbs cyberattack to russi...,[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
297,https://www.msnbc.com/transcripts/transcript-r...,2021-06-02 01:00:00+00:00,SummaryPresident Biden spoke in Oklahoma today...,summarypresident biden spoke in oklahoma today...,[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
298,https://www.msnbc.com/transcripts/transcript-r...,2021-05-28 01:00:00+00:00,SummaryRepublican lawmakers in the Texas state...,summaryrepublican lawmakers in the texas state...,[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


In [16]:
#find all matches in each Tucker transcript for each word in the primary abortion issue dicationary and extend any matches onto the empty list under
#the 'abortion' column for that transcript (row). 
def find_matches(data2, pew):

    #Abortion words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][0]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Abortion'][count].extend(match)   

    #Education words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][1]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Education'][count].extend(match)   


    #Immigration words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][2]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Immigration'][count].extend(match)        


    #Violent crime words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][3]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Violent crime'][count].extend(match)     


    #Health care words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][4]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Health care'][count].extend(match) 


    #Gun policy words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][5]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Gun policy'][count].extend(match)   


    #Energy policy words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][6]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Energy policy'][count].extend(match)  


    #Economy words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][7]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Economy'][count].extend(match)     


    #Issues around race and ethnicity words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][8]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Issues around race and ethnicity'][count].extend(match)


    #Supreme Court appointments words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][9]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Supreme Court appointments'][count].extend(match)


    #Size and scope of federal government words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][10]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Size and scope of federal government'][count].extend(match)


    #Climate change words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][11]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Climate change'][count].extend(match)

     #Voting policies words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][12]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Voting policies'][count].extend(match)


    #Foreign policy words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][13]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Foreign policy'][count].extend(match)


    #The coronavirus outbreak words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][14]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['The coronavirus outbreak'][count].extend(match)  


    #Future of democracy 
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][15]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Future of Democracy in the country'][count].extend(match)

    #Investigations into Trump words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][16]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Investigations into Trump'][count].extend(match)


    #Investigations into Biden words
    for count, dfrow in enumerate(data2['text_processed']):
            for word in pew['primary dict'][17]:
                match = re.findall(fr'\b{word}\b',dfrow)
                data2['Investigations into Biden'][count].extend(match)

    return data2

In [17]:
data2_tucker = find_matches(data_tucker, pew)

#data2_tucker

In [18]:
data2_maddow = find_matches(data_maddow, pew)


In [19]:
binary_tucker=data2_tucker.iloc[:,2:].copy()
binary_maddow=data2_maddow.iloc[:,2:].copy()


In [20]:
binary_tucker

,title,text,transcript,text_processed,Abortion,Education,Immigration,Violent crime,Health care,Gun policy,...,Issues around race and ethnicity,Supreme Court appointments,Size and scope of federal government,Climate change,Voting policies,Foreign policy,The coronavirus outbreak,Future of Democracy in the country,Investigations into Trump,Investigations into Biden
0,Tucker Carlson: Socialism with open borders is...,Democrats want to eliminate restrictions on im...,Democrats want to eliminate restrictions on im...,democrats want to eliminate restrictions on im...,[],[],"[immigration, immigration, immigration, immigr...",[crime],[health care],[],...,[],[],[],[],[],[war],[],[],[],[]
1,"Tucker Carlson: For the crime of forgetting, J...",Jerome Corsi addresses charges against him in ...,Jerome Corsi addresses charges against him in ...,jerome corsi addresses charges against him in ...,[],[],[],[crime],[],[],...,[],[],[],[],[],[],[],[],"[mueller, mueller, mueller, mueller]","[laptop, laptop, laptop]"
2,Tucker Carlson: The word 'Russia' eliminates a...,"On Wednesday, Tucker Carlson exposes the flaws...","On Wednesday, Tucker Carlson exposes the flaws...",on wednesday tucker carlson exposes the flaws ...,[],[],[immigration],[],[],[],...,[],[],[],[],[],[],[],[],[mueller],[]
3,Tucker Carlson: The Mueller probe and what we ...,Michael Cohen pleads guilty to lying to Congre...,Michael Cohen pleads guilty to lying to Congre...,michael cohen pleads guilty to lying to congre...,[],[],[],"[rule of law, rule of law, law and order]",[],[],...,[],[],[],[],[],[war],[],[],"[trump investigation, trump investigation]",[]
4,Tucker Carlson: The Mueller probe continues to...,"On Monday, Tucker Carlson talks to Jerome Cors...","On Monday, Tucker Carlson talks to Jerome Cors...",on monday tucker carlson talks to jerome corsi...,[],[],[],[],[],[],...,[],[],[],[],[],[putin],[],[],"[mueller, mueller, mueller, mueller, mueller]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,TUCKER CARLSON: The lies 'just kept coming' du...,Fox News host Tucker Carlson reacts to Presid...,Fox News host Tucker Carlson reacts to Presid...,fox news host tucker carlson reacts to presid...,"[abortion, abortion, abortion, abortion, abort...","[education, schools, school, school, teachers]",[],[],[],[],...,"[blm, racism]",[],[],[],[],"[nato, nato, putin, xi, jinping]",[],"[january 6, insurrection]",[],[hunter biden]
809,TUCKER CARLSON: Self-defense is becoming illeg...,WARNING: Graphic footage—Fox News host Tucker ...,WARNING: Graphic footage—Fox News host Tucker ...,warning: graphic footage—fox news host tucker ...,[],[],[],"[crime, crime, crime, criminals, theft, violen...",[],[],...,"[blm, blm, blm, blm, social justice, unarmed]",[],[],[],[],[],[],[],[],[]
810,"TUCKER CARLSON: Food, water, energy and infras...",Fox News host Tucker Carlson reacts to leaders...,Fox News host Tucker Carlson reacts to leaders...,fox news host tucker carlson reacts to leaders...,[],[],[],[],[],[],...,"[racism, racism]",[],[],[climate change],[],[war],[],[],[],[]
811,TUCKER CARLSON: Mayor Pete is completely incom...,Fox News host Tucker Carlson calls out the Bid...,Fox News host Tucker Carlson calls out the Bid...,fox news host tucker carlson calls out the bid...,[],[],[],[],[],[],...,"[equity, equity]",[],[],"[climate change, climate change]",[],[war],[],[],[],[]


In [21]:
def nonzero(x):
    if len(x) > 0:
        return 1
    else:
        return 0

In [22]:
binary_tucker = binary_tucker.applymap(nonzero)
binary_maddow = binary_maddow.applymap(nonzero)

In [23]:

#binary_tucker['timestamp'] = data2_tucker['timestamp']
#binary_maddow['timestamp'] = data2_maddow['timestamp']

binary_tucker['timestamp']  = pd.to_datetime(data2_tucker['timestamp'])
binary_maddow['timestamp'] = pd.to_datetime(data2_maddow['timestamp']).dt.tz_convert(None)

In [43]:
binary_maddow

,transcript,text_processed,Abortion,Education,Immigration,Violent crime,Health care,Gun policy,Energy policy,Economy,...,Size and scope of federal government,Climate change,Voting policies,Foreign policy,The coronavirus outbreak,Future of Democracy in the country,Investigations into Trump,Investigations into Biden,timestamp,host
0,1,1,1,1,0,1,0,1,0,0,...,0,0,0,1,1,1,0,0,2022-10-04 01:00:00,Rachel Maddow
1,1,1,1,0,0,1,0,0,0,1,...,0,0,1,1,0,1,0,0,2022-09-27 01:00:00,Rachel Maddow
2,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,2022-09-13 01:00:00,Rachel Maddow
3,1,1,0,1,0,1,0,0,1,1,...,0,0,0,1,1,0,0,0,2022-08-30 01:00:00,Rachel Maddow
4,1,1,0,0,0,1,0,0,0,1,...,0,0,0,1,1,1,1,0,2022-08-23 01:00:00,Rachel Maddow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,1,0,1,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,2021-06-04 01:00:00,Rachel Maddow
296,1,1,0,0,0,1,1,0,0,1,...,0,0,1,1,1,0,0,0,2021-06-03 01:00:00,Rachel Maddow
297,1,1,0,0,1,1,0,0,1,1,...,0,0,1,1,1,1,0,0,2021-06-02 01:00:00,Rachel Maddow
298,1,1,0,1,0,0,0,0,0,1,...,0,0,1,1,1,1,0,0,2021-05-28 01:00:00,Rachel Maddow


In [44]:
grouped_tucker= binary_tucker.groupby(pd.Grouper(key='timestamp', axis=0, 
                      freq='M')).sum().drop(columns=['title','text','text_processed'])

grouped_maddow= binary_maddow.groupby(pd.Grouper(key='timestamp', axis=0, 
                      freq='M')).sum().drop(columns=['text_processed'])


In [45]:
# grouped_maddow

In [46]:
#grouped_tucker
# def normalize(df):  
#     df.iloc[:,1:] = df.iloc[:,1:].div(df.transcript, axis=0)
#     return df


In [47]:
# normalize_tucker = normalize(grouped_tucker)
# normalize_maddow = normalize(grouped_maddow)
grouped_tucker['host']='Tucker Carlson'
grouped_maddow['host']='Rachel Maddow'

In [48]:
final = pd.concat([grouped_tucker, grouped_maddow]).reset_index().sort_values(by='timestamp').drop(columns=['transcript']).dropna()
valvars = final.columns[1:-1].to_list()    
valvars

['Abortion',
 'Education',
 'Immigration',
 'Violent crime',
 'Health care',
 'Gun policy',
 'Energy policy',
 'Economy',
 'Issues around race and ethnicity',
 'Supreme Court appointments',
 'Size and scope of federal government',
 'Climate change',
 'Voting policies',
 'Foreign policy',
 'The coronavirus outbreak',
 'Future of Democracy in the country',
 'Investigations into Trump',
 'Investigations into Biden']

In [49]:
# valuevars=final['timestamp'].to_list()


In [50]:
# use pd.melt to get the data into "long form" for they dynamic altair viz
changedata = pd.melt(final, id_vars=['host','timestamp'], value_vars=['Abortion',
 'Education',
 'Immigration',
 'Violent crime',
 'Health care',
 'Gun policy',
 'Energy policy',
 'Economy',
 'Issues around race and ethnicity',
 'Supreme Court appointments',
 'Size and scope of federal government',
 'Climate change',
 'Voting policies',
 'Foreign policy',
 'The coronavirus outbreak',
 'Future of Democracy in the country',
 'Investigations into Trump',
 'Investigations into Biden'])
changedata

,host,timestamp,variable,value
0,Tucker Carlson,2018-11-30,Abortion,0
1,Tucker Carlson,2018-12-31,Abortion,0
2,Tucker Carlson,2019-01-31,Abortion,1
3,Tucker Carlson,2019-02-28,Abortion,5
4,Tucker Carlson,2019-03-31,Abortion,0
...,...,...,...,...
1255,Tucker Carlson,2022-10-31,Investigations into Biden,2
1256,Tucker Carlson,2022-11-30,Investigations into Biden,1
1257,Tucker Carlson,2022-12-31,Investigations into Biden,2
1258,Tucker Carlson,2023-01-31,Investigations into Biden,5


In [51]:
#Generate dynamic Altair chart


def generateLineChartP21(changedata):

    #dropdown options for sorting data by indicator
    metricOptions = ['Tucker Carlson',
       'Rachel Maddow']
    
    input_dropdown = None
    
    dropdown_selection = None
    
    #input the dropdown menu and bind it to the dropdown selection that uses the indicators as its field
    input_dropdown = alt.binding_select(options=metricOptions, name="host")
    dropdown_selection = alt.selection_single(fields=['host'], init={"host":metricOptions[0]}, bind=input_dropdown)
    

    #create a base line chart with a tooltip that will show the country name, year, and amount for every point on the graph
    line = alt.Chart(changedata).mark_line().encode( 
         x = 'timestamp:N',
         y = 'value:Q',
         tooltip=['variable:N', 'timestamp:N'],
     
         color = alt.Color('variable', 
                  legend=None)
    )
     
    #make a selection that will highlight individual lines based on the country name for the point that the mouse is nearest to   
    highlight = alt.selection(type='single', on='mouseover',
                          fields=['variable'], nearest=True)
    
    #mark the invisible points adding the highlight selection that determines which field to highlight based on mouseover proximity
    points1 = line.mark_circle().encode(
    opacity=alt.value(0)
    ).add_selection(
    highlight
    )
      
    #line chart that increases the size of the lines of the base line chart based on the condition of whether the mouse is closest
    #to a point on the highlight selection being used on the points
    lines = line.mark_line().encode(
    size=alt.condition(highlight, alt.value(3), alt.value(0.5))
    )
    
    #layer the points and lines charts and add the dropdown selection that filters the data accordingly
    final = alt.layer(
        points1, lines).add_selection(
    dropdown_selection
    ).transform_filter(
    dropdown_selection
    )
    
    return final


In [52]:
generateLineChartP21(changedata)

alt.LayerChart(...)